# 분석 기법 2: 계층적 베이즈 모델을 이용한 시나리오 분석

**연구 목표:** 제한된 실제 시험 데이터(예: 2015년 생산분)에 분석가의 통계적 가정을 결합하여, 전체 이종 비축물량의 신뢰도를 추정한다.

**분석 기법:**
- **계층적 베이즈 모델 (Hierarchical Bayesian Model):** 여러 그룹(생산 연도/롯트) 간의 정보를 공유하여, 데이터가 부족하거나 없는 그룹에 대해서도 합리적인 추론을 가능하게 하는 통계적 프레임워크.
- **사전 확률 분포 (Prior Distribution):** 분석가의 지식이나 가정을 모델에 수학적으로 명시하는 방법. 이 노트북에서는 이 사전 확률을 제어하여 'What-if' 시나리오 분석을 수행한다.

**핵심 아이디어:**
"만약 생산 연도별 성능 차이가 이 정도라고 가정한다면, 전체 신뢰도는 어떻게 추정될 것인가?" 라는 질문에 답한다.

## 1. 환경 설정

분석에 필요한 라이브러리를 불러옵니다.

In [ ]:
import pymc as pm
import pandas as pd
import numpy as np
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 스타일 설정
sns.set_theme(style="whitegrid")
try:
    plt.rcParams['font.family'] = 'Malgun Gothic' # Windows
except:
    plt.rcParams['font.family'] = 'AppleGothic' # Mac
plt.rcParams['axes.unicode_minus'] = False

## 2. 데이터 준비

실제 분석에서는 외부 CSV 파일을 불러오게 됩니다. 여기서는 2025년에 2015년 생산분 3개 롯트를 시험한 상황을 가정한 데이터를 사용합니다.

In [ ]:
data = pd.DataFrame({
    'production_lot': ['2015-LOT-01', '2015-LOT-02', '2015-LOT-04'],
    'num_tested': [11, 10, 12],
    'num_failures': [0, 1, 1]
})
data['num_success'] = data['num_tested'] - data['num_failures']

print("--- 관측 데이터 ---")
print(data)

## 3. 모델링을 위한 인덱스 설정

모델이 전체 비축물량(2015-2019년, 각 6개 롯트)의 구조를 이해할 수 있도록, 각 롯트와 연도에 고유한 숫자 인덱스를 부여합니다.

In [ ]:
all_years = np.arange(2015, 2020)
all_lots = [f"{year}-LOT-{i:02d}" for year in all_years for i in range(1, 7)]

# 각 롯트와 연도에 숫자 인덱스 매핑
lot_map = {lot: i for i, lot in enumerate(all_lots)}
year_map = {year: i for i, year in enumerate(all_years)}

# 각 롯트가 몇 번째 연도에 속하는지에 대한 정보
year_of_lot = np.array([int(lot.split('-')[0]) for lot in all_lots])
year_idx_of_lot = np.array([year_map[y] for y in year_of_lot])

# 관측된 데이터의 롯트 인덱스
observed_lot_idx = [lot_map[lot] for lot in data['production_lot']]

## 4. 시나리오 기반 계층적 베이즈 모델링

분석가의 가정을 모델에 주입하는 핵심 단계입니다. 아래의 `ASSUMPTION CONTROLS` 섹션에서 각 파라미터를 조절하여 다양한 "What-if" 시나리오 분석을 수행할 수 있습니다.

In [ ]:
# ==============================================================================
# === ASSUMPTION CONTROLS (사용자가 직접 제어하는 부분) ===
# ==============================================================================

# 1. 연도별 평균 성능의 분산 (Inter-Year Variance)
#    생산 연도별 평균 신뢰도가 얼마나 다를 수 있는지에 대한 가정입니다.
#    - 작은 값 (예: 0.01): 모든 연도의 성능이 거의 동일할 것이라는 '낙관적' 가정.
#    - 큰 값 (예: 0.2): 연도별 성능 차이가 클 수 있다는 '보수적' 가정.
inter_year_sigma = 0.1

# 2. 연내 개체별 성능의 분산 (Intra-Year/Lot Variance)
#    같은 해에 생산된 롯트들 간의 신뢰도가 얼마나 흩어져 있는지에 대한 가정입니다.
#    - 작은 값 (예: 0.02): 한 해의 생산품들은 품질이 매우 균일하다는 가정.
#    - 큰 값 (예: 0.1): 롯트별로 품질 편차가 있을 수 있다는 가정.
intra_lot_sigma = 0.05

# 3. 시간에 따른 분산 변화 (Degradation Effect on Variance)
#    저장 기간이 길어질수록 성능의 일관성이 떨어져 분산이 증가하는 효과를 모델에 포함할지 여부입니다.
#    - True: 시간에 따라 분산이 증가하는 '열화 효과'를 고려합니다. (더 보수적)
#    - False: 분산은 시간에 관계없이 일정하다고 가정합니다.
degradation_effect_on_variance = True

print("--- 분석 가정 설정 ---")
print(f"> 연도별 분산 가정 (sigma): {inter_year_sigma}")
print(f"> 롯트별 분산 가정 (sigma): {intra_lot_sigma}")
print(f"> 시간에 따른 분산 증가 효과: {'고려함' if degradation_effect_on_variance else '고려 안함'}")

with pm.Model() as hierarchical_model:
    # === 1. 사전 확률 (Priors) ===
    # 전체 평균 신뢰도
    mu_global_logit = pm.Normal('mu_global_logit', mu=3.89, sigma=0.5) # 평균 98% 신뢰도(logit(0.98)≈3.89)를 중심으로 가정

    # 연도별 변동성 (가정 주입)
    sigma_year = pm.HalfNormal('sigma_year', sigma=inter_year_sigma)

    # 롯트별 변동성 (연내 변동성)
    sigma_lot_base = pm.HalfNormal('sigma_lot_base', sigma=intra_lot_sigma)

    # 시간에 따른 분산 증가 효과 (가정 주입)
    if degradation_effect_on_variance:
        # 시간이 지날수록 분산이 커지는 효과를 모델링하는 파라미터
        # 0보다 큰 값을 가지며, 평균적으로 작은 효과를 가질 것으로 가정
        variance_degradation_rate = pm.HalfNormal('variance_degradation_rate', sigma=0.05)
        # 각 롯트의 나이(2025년 기준) 계산
        age_of_lot = 2025 - year_of_lot
        # 최종 롯트별 분산 = 기본 분산 + (나이 * 분산 증가율)
        sigma_lot_effective = pm.Deterministic('sigma_lot_effective', sigma_lot_base + age_of_lot * variance_degradation_rate)
    else:
        sigma_lot_effective = pm.Deterministic('sigma_lot_effective', sigma_lot_base)

    # === 2. 계층 구조 ===
    # 각 연도의 신뢰도 (Logit 스케일)
    theta_year = pm.Normal('theta_year', mu=mu_global_logit, sigma=sigma_year, shape=len(all_years))

    # 각 롯트의 신뢰도 (Logit 스케일)
    # 분산이 롯트별로 다를 수 있으므로, sigma 인자에 배열을 전달
    theta_lot = pm.Normal('theta_lot', mu=theta_year[year_idx_of_lot], sigma=sigma_lot_effective, shape=len(all_lots))
    
    # 실제 신뢰도 (0과 1 사이 값)
    reliability_lot = pm.Deterministic('reliability_lot', pm.invlogit(theta_lot))

    # === 3. 가능도 (Likelihood) ===
    # 모델과 관측 데이터 연결
    y_obs = pm.Binomial(
        'y_obs',
        n=data['num_tested'].values,
        p=reliability_lot[observed_lot_idx],
        observed=data['num_success'].values
    )

    # === 4. MCMC 샘플링 ===
    trace = pm.sample(2000, tune=1500, cores=1, return_inferencedata=True, random_seed=2024)

## 5. 결과 분석 및 시각화

모델이 추정한 모든 롯트(시험한 롯트와 시험하지 않은 롯트 모두)의 신뢰도 분포를 확인하고 시각화합니다.

In [ ]:
# 포레스트 플롯(Forest Plot)으로 시각화
fig, ax = plt.subplots(figsize=(10, 12))

az.plot_forest(
    trace, 
    var_names=['reliability_lot'], 
    combined=True, 
    hdi_prob=0.9, # 90% 신뢰구간(HDI)
    ax=ax
)

ax.set_yticklabels(all_lots[::-1]) # y축 레이블을 롯트 이름으로 변경
ax.set_title(f'시나리오 분석: 롯트별 신뢰도 추정 (90% HDI)', fontsize=16)
ax.vlines(pm.invlogit(trace.posterior['mu_global_logit']).mean(), *ax.get_ylim(), 
          linestyle='--', color='red', label='전체 평균 신뢰도(추정치)')
ax.legend()
plt.show()

### 전체 비축물량 신뢰도 분포

모든 롯트의 신뢰도 추정치를 평균내어, 전체 비축물량의 평균 신뢰도 분포를 확인합니다.

In [ ]:
# trace에서 모든 롯트의 신뢰도 샘플을 추출하여 평균 계산
posterior_samples = trace.posterior['reliability_lot'].values
mean_stockpile_reliability = posterior_samples.mean(axis=2) # 롯트 축에 대해 평균

# 분포 시각화
plt.figure(figsize=(10, 6))
sns.histplot(mean_stockpile_reliability.flatten(), kde=True, bins=50, stat='density')
plt.title(f'시나리오 분석: 전체 비축물량 평균 신뢰도 분포', fontsize=16)
plt.xlabel('전체 평균 신뢰도')
plt.ylabel('빈도')

# 90% 신뢰구간(HDI) 계산 및 표시
hdi = az.hdi(mean_stockpile_reliability, hdi_prob=0.9)
plt.axvline(hdi[0], color='red', linestyle='--', label=f'90% HDI: [{hdi[0]:.3f}, {hdi[1]:.3f}]')
plt.axvline(hdi[1], color='red', linestyle='--')
plt.legend()
plt.show()

print(f"90% 신뢰수준에서 전체 비축물량의 평균 신뢰도는 최소 {hdi[0]:.3f} 이상일 것으로 추정됩니다.")